In [1]:
# Importing the required libraries
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from functools import partial

In [4]:
from dotenv import load_dotenv

from pathlib import Path

env_path = Path("../../.env-live")

if env_path.exists():
    print('envs Loaded')
    load_dotenv(dotenv_path=env_path)
from jrjModelRegistry.jrjModelRegistry import registerAJrjModel

envs Loaded
Pinged your deployment. You successfully connected to MongoDB!


In [5]:
def generalDtPredictor(self, transformedData):
    import pandas as pd
    probs = self.predict_proba(transformedData)
    return pd.Series(probs[:, 1])

In [6]:
def generalDtTransformer(dataForTransfer = None):
    import pandas as pd
    if isinstance(dataForTransfer, pd.DataFrame):
        df = dataForTransfer.copy()
    else:
        df = pd.DataFrame(dataForTransfer)
    return df

In [7]:
# Reading the csv file and putting it into 'df' object.
df = pd.read_csv('heart_v2.csv')

In [8]:
df.columns

Index(['age', 'sex', 'BP', 'cholestrol', 'heart disease'], dtype='object')

In [9]:
df.head()

,age,sex,BP,cholestrol,heart disease
0,70,1,130,322,1
1,67,0,115,564,0
2,57,1,124,261,1
3,64,1,128,263,0
4,74,0,120,269,0


In [10]:
hDSampleData = {
    "age": [70],
    "sex": [1],
    "BP": [130],
    "cholestrol": [322],
}
hDSampleData

{'age': [70], 'sex': [1], 'BP': [130], 'cholestrol': [322]}

In [11]:
# Putting feature variable to X
X = df.drop('heart disease',axis=1)

# Putting response variable to y
y = df['heart disease']

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((189, 4), (81, 4))

Fitting the decision tree with default hyperparameters, apart from max_depth which is 3 so that we can plot and read the tree.

In [14]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
dt = DecisionTreeClassifier(max_depth=3)
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3)

In [16]:
!pip install six

In [17]:
# Importing required packages for visualization
from IPython.display import Image  
from six import StringIO  
from sklearn.tree import export_graphviz
import pydotplus, graphviz

In [18]:
# plotting tree with max_depth=3
dot_data = StringIO()  

export_graphviz(dt, out_file=dot_data, filled=True, rounded=True,
                feature_names=X.columns, 
                class_names=['No Disease', "Disease"])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())
#Image(graph.create_png(),width=800,height=900)
#graph.write_pdf("dt_heartdisease.pdf")

InvocationException: GraphViz's executables not found

#### Evaluating model performance

In [19]:
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [21]:
print(accuracy_score(y_train, y_train_pred))
confusion_matrix(y_train, y_train_pred)

0.7407407407407407


array([[82, 19],
       [30, 58]])

In [22]:
print(accuracy_score(y_test, y_test_pred))
confusion_matrix(y_test, y_test_pred)

0.6049382716049383


array([[35, 14],
       [18, 14]])

Creating helper functions to evaluate model performance and help plot the decision tree

In [23]:
def get_dt_graph(dt_classifier):
    dot_data = StringIO()
    export_graphviz(dt_classifier, out_file=dot_data, filled=True,rounded=True,
                    feature_names=X.columns, 
                    class_names=['Disease', "No Disease"])
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    return graph

In [24]:
def evaluate_model(dt_classifier):
    print("Train Accuracy :", accuracy_score(y_train, dt_classifier.predict(X_train)))
    print("Train Confusion Matrix:")
    print(confusion_matrix(y_train, dt_classifier.predict(X_train)))
    print("-"*50)
    print("Test Accuracy :", accuracy_score(y_test, dt_classifier.predict(X_test)))
    print("Test Confusion Matrix:")
    print(confusion_matrix(y_test, dt_classifier.predict(X_test)))

### Without setting any hyper-parameters

In [25]:
dt_default = DecisionTreeClassifier(random_state=42)
dt_default.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [26]:
gph = get_dt_graph(dt_default)
Image(gph.create_png())

InvocationException: GraphViz's executables not found

In [27]:
evaluate_model(dt_default)

Train Accuracy : 1.0
Train Confusion Matrix:
[[101   0]
 [  0  88]]
--------------------------------------------------
Test Accuracy : 0.6296296296296297
Test Confusion Matrix:
[[31 18]
 [12 20]]


In [28]:
from sklearn.metrics import accuracy_score, classification_report
import json

In [29]:
dt_default.transformer = generalDtTransformer
dt_default.mainPredictor = partial(generalDtPredictor,dt_default)

In [30]:
y_pred = dt_default.predict(generalDtTransformer(X_test))
# y_pred
score = accuracy_score(y_test, y_pred)
# score
report = classification_report(y_test, y_pred, output_dict=True)
# report

In [31]:
dt_default_metadata = {
    "modelName": "chenjunxiu_xu__dTModelDefault",
    "version": "1.0.1",
    "params": dt_default.get_params(),  # All model hyperparameters
    "score": float(score),         # Accuracy
    "modelLibrary": "sklearn.tree.DecisionTreeClassifier",
    "libraryMetadata": {
        "feature_importances": dt_default.feature_importances_.tolist(),
        "n_features": int(dt_default.n_features_in_),
        "n_classes": int(dt_default.n_classes_),
        "classes": dt_default.classes_.tolist(),
        "depth": int(dt_default.get_depth()),
        "n_leaves": int(dt_default.get_n_leaves()),
        "classification_report": report
    },
     "sampleData": {
        "dataForTransfer": hDSampleData
    }
}

In [32]:
registerAJrjModel(
    dt_default,
    dt_default_metadata
)

✅ Uploaded encrypted ZIP to s3://273-g4/chenjunxiu_xu__dTModelDefault__1.0.1.pkl.zip


{'_id': '687116ab424cbe18acedce0a',
 'modelName': 'chenjunxiu_xu__dTModelDefault',
 'version': '1.0.1',
 'params': {'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'gini',
  'max_depth': None,
  'max_features': None,
  'max_leaf_nodes': None,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'monotonic_cst': None,
  'random_state': 42,
  'splitter': 'best'},
 'score': 0.6296296296296297,
 'modelLibrary': 'sklearn.tree.DecisionTreeClassifier',
 'libraryMetadata': {'feature_importances': [0.281150807488585,
   0.10487821011991588,
   0.24672499344606771,
   0.3672459889454313],
  'n_features': 4,
  'n_classes': 2,
  'classes': [0, 1],
  'depth': 15,
  'n_leaves': 58,
  'classification_report': {'0': {'precision': 0.7209302325581395,
    'recall': 0.6326530612244898,
    'f1-score': 0.6739130434782609,
    'support': 49.0},
   '1': {'precision': 0.5263157894736842,
    'recall': 0.625,
    'f1-score': 0.5714285

### Controlling the depth of the tree

In [33]:
?DecisionTreeClassifier

Init signature:
DecisionTreeClassifier(
    *,
    criterion='gini',
    splitter='best',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    class_weight=None,
    ccp_alpha=0.0,
    monotonic_cst=None,
)
Docstring:     
A decision tree classifier.

Read more in the :ref:`User Guide <tree>`.

Parameters
----------
criterion : {"gini", "entropy", "log_loss"}, default="gini"
    The function to measure the quality of a split. Supported criteria are
    "gini" for the Gini impurity and "log_loss" and "entropy" both for the
    Shannon information gain, see :ref:`tree_mathematical_formulation`.

splitter : {"best", "random"}, default="best"
    The strategy used to choose the split at each node. Supported
    strategies are "best" to choose the best split and "random" to choose
    the best random split.

max_depth : int, default=None

In [34]:
dt_depth = DecisionTreeClassifier(max_depth=3)
dt_depth.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3)

In [35]:
gph = get_dt_graph(dt_depth) 
Image(gph.create_png())

InvocationException: GraphViz's executables not found

In [36]:
evaluate_model(dt_depth)

Train Accuracy : 0.7407407407407407
Train Confusion Matrix:
[[82 19]
 [30 58]]
--------------------------------------------------
Test Accuracy : 0.6049382716049383
Test Confusion Matrix:
[[35 14]
 [18 14]]


In [37]:
dt_depth.transformer = generalDtTransformer
dt_depth.mainPredictor = partial(generalDtPredictor,dt_depth)
y_pred = dt_depth.predict(generalDtTransformer(X_test))
# y_pred
score = accuracy_score(y_test, y_pred)
# score
report = classification_report(y_test, y_pred, output_dict=True)
# report

dt_depth_metadata = {
    "modelName": "chenjunxiu_xu__dTModelDepth",
    "version": "1.0.1",
    "params": dt_depth.get_params(),  # All model hyperparameters
    "score": float(score),         # Accuracy
    "modelLibrary": "sklearn.tree.DecisionTreeClassifier",
    "libraryMetadata": {
        "feature_importances": dt_depth.feature_importances_.tolist(),
        "n_features": int(dt_depth.n_features_in_),
        "n_classes": int(dt_depth.n_classes_),
        "classes": dt_depth.classes_.tolist(),
        "depth": int(dt_depth.get_depth()),
        "n_leaves": int(dt_depth.get_n_leaves()),
        "classification_report": report
    },
     "sampleData": {
        "dataForTransfer": hDSampleData
    }
}
registerAJrjModel(
    dt_depth,
    dt_depth_metadata
)

✅ Uploaded encrypted ZIP to s3://273-g4/chenjunxiu_xu__dTModelDepth__1.0.1.pkl.zip


{'_id': '687116bc424cbe18acedce0b',
 'modelName': 'chenjunxiu_xu__dTModelDepth',
 'version': '1.0.1',
 'params': {'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'gini',
  'max_depth': 3,
  'max_features': None,
  'max_leaf_nodes': None,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'monotonic_cst': None,
  'random_state': None,
  'splitter': 'best'},
 'score': 0.6049382716049383,
 'modelLibrary': 'sklearn.tree.DecisionTreeClassifier',
 'libraryMetadata': {'feature_importances': [0.4094009169839831,
   0.35652876012026785,
   0.12527576387474865,
   0.1087945590210004],
  'n_features': 4,
  'n_classes': 2,
  'classes': [0, 1],
  'depth': 3,
  'n_leaves': 8,
  'classification_report': {'0': {'precision': 0.660377358490566,
    'recall': 0.7142857142857143,
    'f1-score': 0.6862745098039216,
    'support': 49.0},
   '1': {'precision': 0.5,
    'recall': 0.4375,
    'f1-score': 0.4666666666666667,
    'sup

### Specifying minimum samples before split

In [38]:
dt_min_split = DecisionTreeClassifier(min_samples_split=20)
dt_min_split.fit(X_train, y_train)

DecisionTreeClassifier(min_samples_split=20)

In [39]:
gph = get_dt_graph(dt_min_split) 
Image(gph.create_png())

InvocationException: GraphViz's executables not found

In [40]:
evaluate_model(dt_min_split)

Train Accuracy : 0.8359788359788359
Train Confusion Matrix:
[[85 16]
 [15 73]]
--------------------------------------------------
Test Accuracy : 0.6419753086419753
Test Confusion Matrix:
[[32 17]
 [12 20]]


In [41]:
dt_min_split.transformer = generalDtTransformer
dt_min_split.mainPredictor = partial(generalDtPredictor,dt_min_split)
y_pred = dt_min_split.predict(generalDtTransformer(X_test))
# y_pred
score = accuracy_score(y_test, y_pred)
# score
report = classification_report(y_test, y_pred, output_dict=True)
# report

dt_min_split_metadata = {
    "modelName": "chenjunxiu_xu__dTModelMinSplit",
    "version": "1.0.1",
    "params": dt_min_split.get_params(),  # All model hyperparameters
    "score": float(score),         # Accuracy
    "modelLibrary": "sklearn.tree.DecisionTreeClassifier",
    "libraryMetadata": {
        "feature_importances": dt_min_split.feature_importances_.tolist(),
        "n_features": int(dt_min_split.n_features_in_),
        "n_classes": int(dt_min_split.n_classes_),
        "classes": dt_min_split.classes_.tolist(),
        "depth": int(dt_min_split.get_depth()),
        "n_leaves": int(dt_min_split.get_n_leaves()),
        "classification_report": report
    },
     "sampleData": {
        "dataForTransfer": hDSampleData
    }
}
registerAJrjModel(
    dt_min_split,
    dt_min_split_metadata
)

✅ Uploaded encrypted ZIP to s3://273-g4/chenjunxiu_xu__dTModelMinSplit__1.0.1.pkl.zip


{'_id': '687116cf424cbe18acedce0c',
 'modelName': 'chenjunxiu_xu__dTModelMinSplit',
 'version': '1.0.1',
 'params': {'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'gini',
  'max_depth': None,
  'max_features': None,
  'max_leaf_nodes': None,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 1,
  'min_samples_split': 20,
  'min_weight_fraction_leaf': 0.0,
  'monotonic_cst': None,
  'random_state': None,
  'splitter': 'best'},
 'score': 0.6419753086419753,
 'modelLibrary': 'sklearn.tree.DecisionTreeClassifier',
 'libraryMetadata': {'feature_importances': [0.34050057684806834,
   0.1970366925967589,
   0.20374666487365756,
   0.25871606568151534],
  'n_features': 4,
  'n_classes': 2,
  'classes': [0, 1],
  'depth': 9,
  'n_leaves': 21,
  'classification_report': {'0': {'precision': 0.7272727272727273,
    'recall': 0.6530612244897959,
    'f1-score': 0.6881720430107527,
    'support': 49.0},
   '1': {'precision': 0.5405405405405406,
    'recall': 0.625,
    'f1-score': 0.57

### Specifying minimum samples in leaf node

In [42]:
dt_min_leaf = DecisionTreeClassifier(min_samples_leaf=20, random_state=42)
dt_min_leaf.fit(X_train, y_train)

DecisionTreeClassifier(min_samples_leaf=20, random_state=42)

In [43]:
gph = get_dt_graph(dt_min_leaf)
Image(gph.create_png())

InvocationException: GraphViz's executables not found

In [44]:
evaluate_model(dt_min_leaf)

Train Accuracy : 0.7037037037037037
Train Confusion Matrix:
[[85 16]
 [40 48]]
--------------------------------------------------
Test Accuracy : 0.6419753086419753
Test Confusion Matrix:
[[38 11]
 [18 14]]


In [45]:
dt_min_leaf.transformer = generalDtTransformer
dt_min_leaf.mainPredictor = partial(generalDtPredictor,dt_min_leaf)
y_pred = dt_min_leaf.predict(generalDtTransformer(X_test))
# y_pred
score = accuracy_score(y_test, y_pred)
# score
report = classification_report(y_test, y_pred, output_dict=True)
# report

dt_min_leaf_metadata = {
    "modelName": "chenjunxiu_xu__dTModelMinLeaf",
    "version": "1.0.1",
    "params": dt_min_leaf.get_params(),  # All model hyperparameters
    "score": float(score),         # Accuracy
    "modelLibrary": "sklearn.tree.DecisionTreeClassifier",
    "libraryMetadata": {
        "feature_importances": dt_min_leaf.feature_importances_.tolist(),
        "n_features": int(dt_min_leaf.n_features_in_),
        "n_classes": int(dt_min_leaf.n_classes_),
        "classes": dt_min_leaf.classes_.tolist(),
        "depth": int(dt_min_leaf.get_depth()),
        "n_leaves": int(dt_min_leaf.get_n_leaves()),
        "classification_report": report
    },
     "sampleData": {
        "dataForTransfer": hDSampleData
    }
}
registerAJrjModel(
    dt_min_leaf,
    dt_min_leaf_metadata
)

✅ Uploaded encrypted ZIP to s3://273-g4/chenjunxiu_xu__dTModelMinLeaf__1.0.1.pkl.zip


{'_id': '687116dd424cbe18acedce0d',
 'modelName': 'chenjunxiu_xu__dTModelMinLeaf',
 'version': '1.0.1',
 'params': {'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'gini',
  'max_depth': None,
  'max_features': None,
  'max_leaf_nodes': None,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 20,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'monotonic_cst': None,
  'random_state': 42,
  'splitter': 'best'},
 'score': 0.6419753086419753,
 'modelLibrary': 'sklearn.tree.DecisionTreeClassifier',
 'libraryMetadata': {'feature_importances': [0.4189565577913297,
   0.4334005722906855,
   0.02940135334218933,
   0.11824151657579549],
  'n_features': 4,
  'n_classes': 2,
  'classes': [0, 1],
  'depth': 4,
  'n_leaves': 7,
  'classification_report': {'0': {'precision': 0.6785714285714286,
    'recall': 0.7755102040816326,
    'f1-score': 0.7238095238095238,
    'support': 49.0},
   '1': {'precision': 0.56,
    'recall': 0.4375,
    'f1-score': 0.49122807017543857,
 

### Using Entropy instead of Gini

In [46]:
dt_min_leaf_entropy = DecisionTreeClassifier(min_samples_leaf=20, random_state=42, criterion="entropy")
dt_min_leaf_entropy.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', min_samples_leaf=20,
                       random_state=42)

In [47]:
gph = get_dt_graph(dt_min_leaf_entropy)
Image(gph.create_png())

InvocationException: GraphViz's executables not found

In [48]:
evaluate_model(dt_min_leaf_entropy)

Train Accuracy : 0.7037037037037037
Train Confusion Matrix:
[[85 16]
 [40 48]]
--------------------------------------------------
Test Accuracy : 0.6419753086419753
Test Confusion Matrix:
[[38 11]
 [18 14]]


In [49]:
dt_min_leaf_entropy.transformer = generalDtTransformer
dt_min_leaf_entropy.mainPredictor = partial(generalDtPredictor,dt_min_leaf_entropy)
y_pred = dt_min_leaf_entropy.predict(generalDtTransformer(X_test))
# y_pred
score = accuracy_score(y_test, y_pred)
# score
report = classification_report(y_test, y_pred, output_dict=True)
# report

dt_min_leaf_entropy_metadata = {
    "modelName": "chenjunxiu_xu__dTModelMinLeafEntropy",
    "version": "1.0.1",
    "params": dt_min_leaf_entropy.get_params(),  # All model hyperparameters
    "score": float(score),         # Accuracy
    "modelLibrary": "sklearn.tree.DecisionTreeClassifier",
    "libraryMetadata": {
        "feature_importances": dt_min_leaf_entropy.feature_importances_.tolist(),
        "n_features": int(dt_min_leaf_entropy.n_features_in_),
        "n_classes": int(dt_min_leaf_entropy.n_classes_),
        "classes": dt_min_leaf_entropy.classes_.tolist(),
        "depth": int(dt_min_leaf_entropy.get_depth()),
        "n_leaves": int(dt_min_leaf_entropy.get_n_leaves()),
        "classification_report": report
    },
     "sampleData": {
        "dataForTransfer": hDSampleData
    }
}
registerAJrjModel(
    dt_min_leaf_entropy,
    dt_min_leaf_entropy_metadata
)

✅ Uploaded encrypted ZIP to s3://273-g4/chenjunxiu_xu__dTModelMinLeafEntropy__1.0.1.pkl.zip


{'_id': '687116eb424cbe18acedce0e',
 'modelName': 'chenjunxiu_xu__dTModelMinLeafEntropy',
 'version': '1.0.1',
 'params': {'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'entropy',
  'max_depth': None,
  'max_features': None,
  'max_leaf_nodes': None,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 20,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'monotonic_cst': None,
  'random_state': 42,
  'splitter': 'best'},
 'score': 0.6419753086419753,
 'modelLibrary': 'sklearn.tree.DecisionTreeClassifier',
 'libraryMetadata': {'feature_importances': [0.3779822695721275,
   0.462600993892281,
   0.029843339527990835,
   0.12957339700760065],
  'n_features': 4,
  'n_classes': 2,
  'classes': [0, 1],
  'depth': 4,
  'n_leaves': 7,
  'classification_report': {'0': {'precision': 0.6785714285714286,
    'recall': 0.7755102040816326,
    'f1-score': 0.7238095238095238,
    'support': 49.0},
   '1': {'precision': 0.56,
    'recall': 0.4375,
    'f1-score': 0.4912280701

### Hyper-parameter tuning

In [50]:
dt = DecisionTreeClassifier(random_state=42)

In [51]:
from sklearn.model_selection import GridSearchCV

In [52]:
# Create the parameter grid based on the results of random search 
params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

In [53]:
# grid_search = GridSearchCV(estimator=dt, 
#                            param_grid=params, 
#                            cv=4, n_jobs=-1, verbose=1, scoring = "f1")

In [54]:
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=dt, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

In [55]:
%%time
grid_search.fit(X_train, y_train)

Fitting 4 folds for each of 50 candidates, totalling 200 fits
CPU times: user 149 ms, sys: 95.3 ms, total: 244 ms
Wall time: 1.85 s


GridSearchCV(cv=4, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 10, 20],
                         'min_samples_leaf': [5, 10, 20, 50, 100]},
             scoring='accuracy', verbose=1)

In [56]:
score_df = pd.DataFrame(grid_search.cv_results_)
score_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003075,0.001341,0.001940,0.000840,gini,2,5,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.708333,0.744681,0.617021,0.531915,0.650488,0.082762,18
1,0.003212,0.001000,0.002226,0.001421,gini,2,10,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.708333,0.744681,0.617021,0.531915,0.650488,0.082762,18
2,0.002995,0.000726,0.002241,0.001027,gini,2,20,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.687500,0.744681,0.617021,0.531915,0.645279,0.079551,20
3,0.001783,0.000568,0.001381,0.000501,gini,2,50,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.666667,0.680851,0.574468,0.702128,0.656028,0.048751,8
4,0.002412,0.001448,0.001246,0.000631,gini,2,100,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.541667,0.531915,0.531915,0.531915,0.534353,0.004223,41


In [57]:
score_df.nlargest(5,"mean_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
30,0.004870,0.004154,0.001822,0.000904,entropy,3,5,"{'criterion': 'entropy', 'max_depth': 3, 'min_...",0.604167,0.765957,0.617021,0.680851,0.666999,0.064090,1
6,0.002185,0.000562,0.001770,0.001021,gini,3,10,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.645833,0.744681,0.595745,0.680851,0.666777,0.054202,2
31,0.001107,0.000118,0.000704,0.000086,entropy,3,10,"{'criterion': 'entropy', 'max_depth': 3, 'min_...",0.625000,0.744681,0.595745,0.680851,0.661569,0.056898,3
5,0.002088,0.001533,0.001462,0.000650,gini,3,5,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.625000,0.744681,0.595745,0.659574,0.656250,0.055831,4
11,0.002135,0.000681,0.001381,0.000839,gini,5,10,"{'criterion': 'gini', 'max_depth': 5, 'min_sam...",0.645833,0.723404,0.595745,0.659574,0.656139,0.045525,5


In [58]:
grid_search.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=5,
                       random_state=42)

In [59]:
dt_best = grid_search.best_estimator_

In [60]:
evaluate_model(dt_best)

Train Accuracy : 0.7195767195767195
Train Confusion Matrix:
[[82 19]
 [34 54]]
--------------------------------------------------
Test Accuracy : 0.6172839506172839
Test Confusion Matrix:
[[36 13]
 [18 14]]


In [61]:
from sklearn.metrics import classification_report

In [62]:
print(classification_report(y_test, dt_best.predict(X_test)))

              precision    recall  f1-score   support

           0       0.67      0.73      0.70        49
           1       0.52      0.44      0.47        32

    accuracy                           0.62        81
   macro avg       0.59      0.59      0.59        81
weighted avg       0.61      0.62      0.61        81



In [63]:
gph = get_dt_graph(dt_best)
Image(gph.create_png())

InvocationException: GraphViz's executables not found

In [64]:
dt_best.transformer = generalDtTransformer
dt_best.mainPredictor = partial(generalDtPredictor,dt_best)
y_pred = dt_best.predict(generalDtTransformer(X_test))
# y_pred
score = accuracy_score(y_test, y_pred)
# score
report = classification_report(y_test, y_pred, output_dict=True)
# report

dt_best_metadata = {
    "modelName": "chenjunxiu_xu__dTModelBestAccuracy",
    "version": "1.0.1",
    "params": dt_best.get_params(),  # All model hyperparameters
    "score": float(score),         # Accuracy
    "modelLibrary": "sklearn.tree.DecisionTreeClassifier",
    "libraryMetadata": {
        "feature_importances": dt_best.feature_importances_.tolist(),
        "n_features": int(dt_best.n_features_in_),
        "n_classes": int(dt_best.n_classes_),
        "classes": dt_best.classes_.tolist(),
        "depth": int(dt_best.get_depth()),
        "n_leaves": int(dt_best.get_n_leaves()),
        "classification_report": report
    },
     "sampleData": {
        "dataForTransfer": hDSampleData
    }
}
registerAJrjModel(
    dt_best,
    dt_best_metadata
)

✅ Uploaded encrypted ZIP to s3://273-g4/chenjunxiu_xu__dTModelBestAccuracy__1.0.1.pkl.zip


{'_id': '68711742424cbe18acedce0f',
 'modelName': 'chenjunxiu_xu__dTModelBestAccuracy',
 'version': '1.0.1',
 'params': {'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'entropy',
  'max_depth': 3,
  'max_features': None,
  'max_leaf_nodes': None,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 5,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'monotonic_cst': None,
  'random_state': 42,
  'splitter': 'best'},
 'score': 0.6172839506172839,
 'modelLibrary': 'sklearn.tree.DecisionTreeClassifier',
 'libraryMetadata': {'feature_importances': [0.487423469019835,
   0.3626657705759344,
   0.0,
   0.1499107604042306],
  'n_features': 4,
  'n_classes': 2,
  'classes': [0, 1],
  'depth': 3,
  'n_leaves': 8,
  'classification_report': {'0': {'precision': 0.6666666666666666,
    'recall': 0.7346938775510204,
    'f1-score': 0.6990291262135923,
    'support': 49.0},
   '1': {'precision': 0.5185185185185185,
    'recall': 0.4375,
    'f1-score': 0.4745762711864407,
  

# tune for f1


In [65]:
grid_search = GridSearchCV(estimator=dt, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "f1")

In [66]:
%%time
grid_search.fit(X_train, y_train)

Fitting 4 folds for each of 50 candidates, totalling 200 fits
CPU times: user 86.8 ms, sys: 16.3 ms, total: 103 ms
Wall time: 193 ms


GridSearchCV(cv=4, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 10, 20],
                         'min_samples_leaf': [5, 10, 20, 50, 100]},
             scoring='f1', verbose=1)

In [67]:
score_df = pd.DataFrame(grid_search.cv_results_)
score_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009748,0.005637,0.009411,0.006136,gini,2,5,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.666667,0.684211,0.500000,0.421053,0.567982,0.111195,27
1,0.009348,0.005643,0.003920,0.000518,gini,2,10,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.666667,0.684211,0.500000,0.421053,0.567982,0.111195,27
2,0.004865,0.003659,0.003771,0.000797,gini,2,20,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.634146,0.684211,0.500000,0.421053,0.559852,0.104681,33
3,0.004376,0.002093,0.002523,0.001002,gini,2,50,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.666667,0.680851,0.565217,0.708333,0.655267,0.054105,1
4,0.002920,0.002153,0.001798,0.000716,gini,2,100,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,41


In [68]:
score_df.nlargest(5,"mean_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
3,0.004376,0.002093,0.002523,0.001002,gini,2,50,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.666667,0.680851,0.565217,0.708333,0.655267,0.054105,1
8,0.001505,0.000902,0.002628,0.001257,gini,3,50,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.666667,0.680851,0.565217,0.708333,0.655267,0.054105,1
13,0.002212,0.001399,0.002062,0.001438,gini,5,50,"{'criterion': 'gini', 'max_depth': 5, 'min_sam...",0.666667,0.680851,0.565217,0.708333,0.655267,0.054105,1
18,0.001832,0.000857,0.002336,0.001340,gini,10,50,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.666667,0.680851,0.565217,0.708333,0.655267,0.054105,1
23,0.001798,0.001039,0.001726,0.001061,gini,20,50,"{'criterion': 'gini', 'max_depth': 20, 'min_sa...",0.666667,0.680851,0.565217,0.708333,0.655267,0.054105,1


In [69]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=2, min_samples_leaf=50, random_state=42)

In [70]:
dt_best = grid_search.best_estimator_

In [71]:
evaluate_model(dt_best)

Train Accuracy : 0.656084656084656
Train Confusion Matrix:
[[62 39]
 [26 62]]
--------------------------------------------------
Test Accuracy : 0.5925925925925926
Test Confusion Matrix:
[[30 19]
 [14 18]]


In [72]:
from sklearn.metrics import classification_report

In [73]:
print(classification_report(y_test, dt_best.predict(X_test)))

              precision    recall  f1-score   support

           0       0.68      0.61      0.65        49
           1       0.49      0.56      0.52        32

    accuracy                           0.59        81
   macro avg       0.58      0.59      0.58        81
weighted avg       0.60      0.59      0.60        81



In [74]:
gph = get_dt_graph(dt_best)
Image(gph.create_png())

InvocationException: GraphViz's executables not found

In [75]:
dt_best.transformer = generalDtTransformer
dt_best.mainPredictor = partial(generalDtPredictor,dt_best)
y_pred = dt_best.predict(generalDtTransformer(X_test))
# y_pred
score = accuracy_score(y_test, y_pred)
# score
report = classification_report(y_test, y_pred, output_dict=True)
# report

dt_best_metadata = {
    "modelName": "chenjunxiu_xu__dTModelBestF1",
    "version": "1.0.1",
    "params": dt_best.get_params(),  # All model hyperparameters
    "score": float(score),         # Accuracy
    "modelLibrary": "sklearn.tree.DecisionTreeClassifier",
    "libraryMetadata": {
        "feature_importances": dt_best.feature_importances_.tolist(),
        "n_features": int(dt_best.n_features_in_),
        "n_classes": int(dt_best.n_classes_),
        "classes": dt_best.classes_.tolist(),
        "depth": int(dt_best.get_depth()),
        "n_leaves": int(dt_best.get_n_leaves()),
        "classification_report": report
    },
     "sampleData": {
        "dataForTransfer": hDSampleData
    }
}
registerAJrjModel(
    dt_best,
    dt_best_metadata
)

✅ Uploaded encrypted ZIP to s3://273-g4/chenjunxiu_xu__dTModelBestF1__1.0.1.pkl.zip


{'_id': '6871175d424cbe18acedce10',
 'modelName': 'chenjunxiu_xu__dTModelBestF1',
 'version': '1.0.1',
 'params': {'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'gini',
  'max_depth': 2,
  'max_features': None,
  'max_leaf_nodes': None,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 50,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'monotonic_cst': None,
  'random_state': 42,
  'splitter': 'best'},
 'score': 0.5925925925925926,
 'modelLibrary': 'sklearn.tree.DecisionTreeClassifier',
 'libraryMetadata': {'feature_importances': [0.9767403709553542,
   0.0,
   0.0,
   0.023259629044645902],
  'n_features': 4,
  'n_classes': 2,
  'classes': [0, 1],
  'depth': 2,
  'n_leaves': 3,
  'classification_report': {'0': {'precision': 0.6818181818181818,
    'recall': 0.6122448979591837,
    'f1-score': 0.6451612903225806,
    'support': 49.0},
   '1': {'precision': 0.4864864864864865,
    'recall': 0.5625,
    'f1-score': 0.5217391304347826,
    'support': 32.0},
